# Udacity Capstone Project - Data Engineering Nanodegree Program

## Cleaning the Data

As mentioned in the README.md file, this project uses two data sources with csv format. These are: 
- _Stock_Countries.csv_
- _Covid_Stringency_Index.csv_

Both files require to fix data quality issues and to be formated to have the structure of our **Staging Tables**.

The Staging tables from which we are going to extract the data for our data model have the next composition:

![alt text](./Data/Img/Staging_tables.PNG)


## STAGING STOCK TABLE

The file "_Stock Countries cleaning_" collects information of more than 1850 companies in 50 different countries around the world. 

Because we want to know the evolution of stock market values throughout the impact of covid-19 we need to collect values of more than a year in order to see the influence of that impact. For that reason I am going to clean up and organize the data to give it a common structure and use the _yfinance_ library to get the values from the stock market during that period of time.


In [1]:
# Import libraries
import configparser

import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

import datetime

import boto3, os

In [2]:
import yfinance as yf
from io import StringIO # python3; python2: BytesIO 

In [3]:
pd.__version__

'1.1.3'

In [4]:
# This is the file we've collected
countries_stock = pd.read_csv("./Data/Stock_Countries.csv", delimiter=";", encoding='utf-8')

First I'm going to check for empty values

In [5]:
#  Check for Nan
countries_stock.isnull().values.any()

True

Because there are empty values I'm going to find how many there are and what values are missing to determine if I should delete faulty data or if I could complete it.

In [6]:
is_NaN = countries_stock.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = countries_stock[row_has_NaN]
rows_with_NaN
# countries_stock['Sector'].isnull().values.any()

,Name,Sector,Ticker,Country
759,NH FOODS LTD,NaN,2282.T,Japan
760,NICHIREI CORP,NaN,2871.T,Japan


Seeing that the data is only missing two cells, I'm going to obtain the information using the _yfinance_ library and complete the data manually.

For that I need to extract the Tickers from this rows and introduce them as parameters in the function _Ticker_ of the _yfinance_ library.

In [7]:
Missing_Sector1_ticker = countries_stock.iloc[rows_with_NaN.index[0]]['Ticker']
Missing_Sector2_ticker = countries_stock.iloc[rows_with_NaN.index[1]]['Ticker']

In [9]:
info_missing1 = yf.Ticker(Missing_Sector1_ticker)
info_missing2 = yf.Ticker(Missing_Sector2_ticker)
sector_missing1 = info_missing1.info['sector']
sector_missing2 = info_missing2.info['sector']
print("Missing sectors: {}, {}".format(sector_missing1, sector_missing2))

Missing sectors: Consumer Defensive, Consumer Defensive


Once determined the missing data we can introduce it into our dataframe.

In [10]:
countries_stock.iloc[rows_with_NaN.index[0]]['Sector'] = sector_missing1
countries_stock.iloc[rows_with_NaN.index[1]]['Sector'] = sector_missing2

Because there were only two cells missing we could do this manually and quickly enough.
If we were missing many values we probably should consider delete these rows or obtain my data from somewhere else to avoid any unneccesary waste of time.

Checking again our data we can see that we have no missing values.

In [11]:
#  Check for Nan
countries_stock.isnull().values.any()

False

### Exploring the data

We can see if we have repeated values and how many companies we have in total.

In [12]:
countries_stock.duplicated(subset = 'Ticker').any()

False

In [13]:
# If there were duplicates
# countries_stock[countries_stock.duplicated(subset = ['Ticker'])]

We can see that our CSV includes 1891 different companies.

In [14]:
# Total number of companies
len(countries_stock)

1891

And 50 different countries.

In [15]:
# The number of countries
len(countries_stock['Country'].unique())

50

In [16]:
# This is the list of the countries of our file
countries_stock['Country'].unique()

array(['Argentina', 'China', 'Singapore', 'Hong Kong', 'South Korea',
       'Taiwan', 'Greece', 'Australia', 'Belgium', 'Brazil', 'Hungary',
       'Canada', 'Chile', 'Colombia', 'Denmark', 'Czech Republic',
       'Spain', 'Estonia', 'Finland', 'France', 'Germany', 'Iceland',
       'Indonesia', 'India', 'Ireland', 'Israel', 'Italy', 'Japan',
       'South Africa', 'Latvia', 'Lithuania', 'Luxembourg', 'Malaysia',
       'Mexico', 'Norway', 'Netherlands', 'New Zealand', 'Peru',
       'Portugal', 'Qatar', 'Romania', 'Russia', 'Sweden', 'Switzerland',
       'Thailand', 'Turkey', 'United Kingdom', 'United States',
       'Venezuela', 'Austria'], dtype=object)

***

Let's check the sectors.

In [17]:
# This are the sectors
# print(countries_stock['Sector'].unique())
print("Number of sectors: {}".format(len(countries_stock['Sector'].unique())))

Number of sectors: 333


Because this file has been created from many different sources we can apreciate that there are too many sectors and there are many repetitions and different formats.

I'm going to reduce this by categories.
We are interested in grouping the companies by a manageable number of sectors, so in order to clean this, I'm going to simplify these sectors to the standard **S&P sectors** based on the company's primary business activity used in the stock market.
The 11 Sectors defined by _The S&P Sectors_ are also used in _Yahoo finances_ and they are: 
- Information Technology
- Health Care
- Financials
- Consumer Discretionary
- Communication Services
- Industrials
- Consumer Staples
- Energy
- Utilities
- Real Estate
- Materials

Here is the definition of each sector and the industries that they include: https://corporatefinanceinstitute.com/resources/knowledge/finance/the-sp-sectors/

In the first place I'm going to create a dictionary containing all the industries of each sector and replacing in the dataframe those values with the name of the sector.

In [18]:
Materials_s = ['Agricultural Inputs',
 'Building Materials',
 'Chemicals',
 'Specialty Chemicals',
 'Lumber & Wood Production',
 'Paper & Paper Products',
 'Aluminum',
 'Copper',
 'Other Industrial Metals & Mining',
 'Gold',
 'Silver',
 'Other Precious Metals & Mining',
 'Coking Coal',
 'Steel']
Materials_d = dict.fromkeys(Materials_s, "Materials")

In [19]:
Financials_s = ['Asset Management',
 'Banks—Diversified',
 'Banks—Regional',
 'Mortgage Finance',
 'Capital Markets',
 'Financial Data & Stock Exchanges',
 'Insurance—Life',
 'Insurance—Property & Casualty',
 'Insurance—Reinsurance',
 'Insurance—Specialty',
 'Insurance Brokers',
 'Insurance—Diversified',
 'Shell Companies',
 'Financial Conglomerates',
 'Credit Services']
Financials_d = dict.fromkeys(Financials_s, "Financials")

In [20]:
Consumer_Defensive_s = ['Beverages—Brewers',
 'Beverages—Wineries & Distilleries',
 'Beverages—Non-Alcoholic',
 'Confectioners',
 'Farm Products',
 'Household & Personal Products',
 'Packaged Foods',
 'Education & Training Services',
 'Discount Stores',
 'Food Distribution',
 'Grocery Stores',
 'Tobacco']
Consumer_Defensive_d = dict.fromkeys(Consumer_Defensive_s, "Consumer Defensive")

In [21]:
Utilities_s = ['Utilities—Independent Power Producers',
 'Utilities—Renewable',
 'Utilities—Regulated Water',
 'Utilities—Regulated Electric',
 'Utilities—Regulated Gas',
 'Utilities—Diversified']
Utilities_d = dict.fromkeys(Utilities_s, "Utilities")

In [22]:
Energy_s = ['Oil & Gas Drilling',
 'Oil & Gas E&P',
 'Oil & Gas Integrated',
 'Oil & Gas Midstream',
 'Oil & Gas Refining & Marketing',
 'Oil & Gas Equipment & Services',
 'Thermal Coal',
 'Uranium']
Energy_d= dict.fromkeys(Energy_s, "Energy")

In [23]:
Technology_s = ['Information Technology Services',
 'Software—Application',
 'Software—Infrastructure',
 'Communication Equipment',
 'Computer Hardware',
 'Consumer Electronics',
 'Electronic Components',
 'Electronics & Computer Distribution',
 'Scientific & Technical Instruments',
 'Semiconductor Equipment & Materials',
 'Semiconductors',
 'Solar']
Technology_d= dict.fromkeys(Technology_s, "Information Technology")

In [24]:
Consumer_Discretionary_s = ['Auto & Truck Dealerships',
 'Auto Manufacturers',
 'Auto Parts',
 'Recreational Vehicles',
 'Furnishings, Fixtures & Appliances',
 'Residential Construction',
 'Textile Manufacturing',
 'Apparel Manufacturing',
 'Footwear & Accessories',
 'Packaging & Containers',
 'Personal Services',
 'Restaurants',
 'Apparel Retail',
 'Department Stores',
 'Home Improvement Retail',
 'Luxury Goods',
 'Internet Retail',
 'Specialty Retail',
 'Gambling',
 'Leisure',
 'Lodging',
 'Resorts & Casinos',
 'Travel Services']
Consumer_Discretionary_d = dict.fromkeys(Consumer_Discretionary_s, "Consumer Discretionary")

In [25]:
Real_Estate_s = ['Real Estate—Development',
 'Real Estate Services',
 'Real Estate—Diversified',
 'REIT—Healthcare Facilities',
 'REIT—Hotel & Motel',
 'REIT—Industrial',
 'REIT—Office',
 'REIT—Residential',
 'REIT—Retail',
 'REIT—Mortgage',
 'REIT—Specialty',
 'REIT—Diversified']
Real_Estate_d = dict.fromkeys(Real_Estate_s, "Real Estate")

In [26]:
Healthcare_s = ['Biotechnology',
 'Drug Manufacturers—General',
 'Drug Manufacturers—Specialty & Generic',
 'Healthcare Plans',
 'Medical Care Facilities',
 'Pharmaceutical Retailers',
 'Health Information Services',
 'Medical Devices',
 'Medical Instruments & Supplies',
 'Diagnostics & Research',
 'Medical Distribution']
Healthcare_d = dict.fromkeys(Healthcare_s, "Healthcare")

In [27]:
Communication_Services_s = ['Telecommunications',
 'Telecom Services',
 'Advertising Agencies',
 'Publishing',
 'Broadcasting',
 'Entertainment',
 'Internet Content & Information',
 'Electronic Gaming & Multimedia']
Communication_Services_d= dict.fromkeys(Communication_Services_s, "Communication Services")

In [28]:
Industrial_s = ['Aerospace & Defense',
 'Specialty Business Services',
 'Consulting Services',
 'Rental & Leasing Services',
 'Security & Protection Services',
 'Staffing & Employment Services',
 'Conglomerates',
 'Engineering & Construction',
 'Infrastructure Operations',
 'Building Products & Equipment',
 'Farm & Heavy Construction Machinery',
 'Industrial Distribution',
 'Business Equipment & Supplies',
 'Specialty Industrial Machinery',
 'Metal Fabrication',
 'Pollution & Treatment Controls',
 'Tools & Accessories',
 'Electrical Equipment & Parts',
 'Airports & Air Services',
 'Airlines',
 'Railroads',
 'Marine Shipping',
 'Trucking',
 'Integrated Freight & Logistics',
 'Waste Management']
Industrial_d= dict.fromkeys(Industrial_s, "Industrials")

In [29]:
# I'll combine all the dictionaries to this one named: "big_dict"
big_dict = {**Industrial_d, **Communication_Services_d, **Healthcare_d, **Real_Estate_d, **Consumer_Discretionary_d, **Technology_d, **Energy_d, **Utilities_d, **Consumer_Defensive_d, **Financials_d, **Materials_d}


In [30]:
# And use it to replace the values in the dataframe
countries_stock["Sector"] = countries_stock["Sector"].replace(big_dict)

In [31]:
# Check the number of sectors again
len(countries_stock["Sector"].unique())

306

We've reduce the number of sectors but it's not nearly enogth.
I'll take a more specific aproach to identify each sector using some key strings that each sector (depending on which category belongs to) has.


In [32]:
# I'm going to create a list with all the sectors and use it to extract the strings
# that contain the key words to identify to which category it belongs.
my_list = countries_stock['Sector'].unique().tolist()
my_list_aux = [each_elem.lower() for each_elem in my_list]

In [33]:
Energy = [x for x in my_list_aux for elem in ['energy', 'gas', 'oil', 'uranium', 'power', 'refining'] if elem in x]
Information_Technology = [x for x in my_list_aux for elem in ['tech', 'software', 'hardware', 'it services','computer', 'electronic', 'cable', 'satellite'] if elem in x]
Health_Care= [x for x in my_list_aux for elem in ['health', 'sanitation', 'pharmac', 'medic'] if elem in x]
Financials = [x for x in my_list_aux for elem in ['financ', 'bank', 'insurance', 'stock', 'asset', 'invest', 'holdings', 'property management'] if elem in x]
Communication_Services = [x for x in my_list_aux for elem in ['communication', 'teleco', 'publish','media', 'internet', 'digital', 'alternative carriers'] if elem in x]
Industrials = [x for x in my_list_aux for elem in ['industri', 'contractor', 'defence', 'defense','aviation','machinery', 'chem', 'cellulose', 'air', 'forest', 'construction', 'transport', 'extraction', 'manufacturing','shipping', 'marine','rental', 'infrastructure', 'leasing', 'agriculture','securi', 'protection', 'employment', 'engineering', 'exploration & production'] if elem in x]
Consumer_Defensive = [x for x in my_list_aux for elem in ['consumer defensive', 'food', 'staples','personal', 'consumer good', 'meat','beverages','drink', 'water', 'tobacco', 'grocery', 'farm products'] if elem in x]
Utilities = [x for x in my_list_aux for elem in ['electri', 'util', 'utilities'] if elem in x]
Real_Estate= [x for x in my_list_aux for elem in ['real estate', 'real state','reit'] if elem in x]
Materials = [x for x in my_list_aux for elem in ['material', 'conglomer', 'wood',  'lumber', 'cement','conductor', 'paper', 'copper', 'chemestry', 'basic resources','metal', 'mining','aluminium', 'gold', 'rubber','silver', 'coal', 'steel'] if elem in x]
Consumer_Discretionary = [x for x in my_list_aux for elem in ['consumer discretionary','auto', 'retail', 'service','wholesale trade', 'private equity','cyclical', 'clothing', 'other products', 'sport', 'apparel','entertainment', 'specialty stores','precision instruments','specialized consumer services', 'mall', 'discount', 'brewers', 'luxury', 'household goods','accessor', 'tourism', 'wines', 'home improvement', 'recreational', 'ceramic', 'watch', 'game', 'gaming', 'casinos', 'restaurant', 'furnish', 'toy', 'texttile', 'footwear', 'accessory', 'shoe', 'gambling', 'travel'] if elem in x]

Now I'll create a new _big_dict_ to replace the values obtained.

In [34]:
Materials_d = dict.fromkeys(Materials, "Materials")
Financials_d = dict.fromkeys(Financials, "Financials")
Consumer_Defensive_d = dict.fromkeys(Consumer_Defensive, "Consumer Defensive")
Utilities_d = dict.fromkeys(Utilities, "Utilities")
Energy_d= dict.fromkeys(Energy, "Energy")
Technology_d= dict.fromkeys(Information_Technology , "Information Technology")
Consumer_Discretionary_d = dict.fromkeys(Consumer_Discretionary, "Consumer Discretionary")
Real_Estate_d = dict.fromkeys(Real_Estate, "Real Estate")
Healthcare_d = dict.fromkeys(Health_Care, "Healthcare")
Communication_Services_d= dict.fromkeys(Communication_Services, "Communication Services")
Industrial_d= dict.fromkeys(Industrials, "Industrials")

big_dict = {**Industrial_d, **Communication_Services_d, **Healthcare_d, **Real_Estate_d, **Consumer_Discretionary_d, **Technology_d, **Energy_d, **Utilities_d, **Consumer_Defensive_d, **Financials_d, **Materials_d}


This time I'll transform the strings of _Sector_ column to lower case to compare it with the values in my dictionary.

In [35]:
countries_stock["Sector"] = countries_stock["Sector"].str.lower()

In [36]:
countries_stock["Sector"] = countries_stock["Sector"].replace(big_dict)

Check the number of sectors again.

In [37]:
countries_stock["Sector"].unique()

array(['Utilities', 'Information Technology', 'Industrials', 'Financials',
       'Consumer Defensive', 'Consumer Discretionary', 'Materials',
       'Communication Services', 'Healthcare', 'Real Estate', 'Energy'],
      dtype=object)

In [38]:
len(countries_stock["Sector"].unique())

11

In [39]:
countries_stock.head()

,Name,Sector,Ticker,Country
0,Compañía de Transporte de Energía Eléctrica en...,Utilities,TRAN.BA,Argentina
1,"Cisco Systems, Inc.",Information Technology,CSCO.BA,Argentina
2,Sociedad Comercial del Plata S.A.,Industrials,COME.BA,Argentina
3,Banco Hipotecario S.A.,Financials,BHIP.BA,Argentina
4,Rigolleau S.A.,Consumer Defensive,RIGO.BA,Argentina


Now that we have our data without missing values and with a simple structure, it's time to use _yfinances_ library to obtain all the stock market values for these companies.

I'm going to set the date values to collect data for a little bit more thatn a year (400 days). That way we'll have enough data to really see the efects of the covid-19 to the stock market.

In [40]:
actual_date = datetime.datetime.today()
start = actual_date - datetime.timedelta(days=401)
end = actual_date - datetime.timedelta(days=1)
# end = datetime.datetime(actual_date.year, actual_date.month, actual_date.day-1)

Download the stock market values for each company.

In [41]:
stocks = countries_stock['Ticker'].to_list()
stock_data = yf.download(stocks, start=start, end=end, group_by='tickers')

[*********************100%***********************]  1891 of 1891 completed

28 Failed downloads:
- LAMDA.AT: No data found for this date range, symbol may be delisted
- OPAP.AT: No data found for this date range, symbol may be delisted
- NELES.HE: No data found for this date range, symbol may be delisted
- PERGY.BD: No data found for this date range, symbol may be delisted
- AEGN.AT: No data found for this date range, symbol may be delisted
- TENERGY.AT: No data found for this date range, symbol may be delisted
- SCH.OL: No data found for this date range, symbol may be delisted
- PPC.AT: No data found for this date range, symbol may be delisted
- EYDAP.AT: No data found for this date range, symbol may be delisted
- TPEIR.AT: No data found for this date range, symbol may be delisted
- ALPHA.AT: No data found for this date range, symbol may be delisted
- ADMIE.AT: No data found for this date range, symbol may be delisted
- BELA.AT: No data found for this date range, symbol may be deliste

We can see in the output that it wasn't possible to obtain the data for some values. The reason for this is that some of the companies didn't have values for the date range given as a parameter (400 days since yesterday).
That's way we can see in the output the next message: "_No data found for this date range, symbol may be delisted_"

We can ignore this. It would work if we'd selected a different date range that contains all the values for every company.

To avoid this message we could delete the companies listed above from our dataframe. I'm going to keep them so I have the chance to use them in case I'd like to use another date range.

__Note: Because the _Tickers_ used by the companies vary with time it can also be possible that the failed downloads are due to the tickers provided in the dataframe in this repository being outdated. 
As of today (27/10/2020) all tickers in the dataframe are valid.__

***

Now I'm going to transform the new dataframe with the stock_data so it has the structure we've described in our Staging tables.

In [42]:
stock_data = stock_data.stack(0).reset_index().rename(columns={'level_1': 'Ticker'})
stock_data = stock_data.melt(id_vars = ['Date', 'Ticker'], value_vars=["Adj Close", "Close", "High", "Low", "Open"], var_name = "Value_type", value_name = "Value")

In [43]:
stock_data.head()

,Date,Ticker,Value_type,Value
0,2019-09-22,ABQK.QA,Adj Close,3.302143
1,2019-09-22,AHCS.QA,Adj Close,0.660800
2,2019-09-22,AKHI.QA,Adj Close,1.762422
3,2019-09-22,ALHE.TA,Adj Close,4814.037109
4,2019-09-22,AMOT.TA,Adj Close,2350.587646


Our data now has a row for each type of value obtained from _yahoo finances_.
We need to add the columns with the sector, name of the company, country and country code that we had in our previous dataframe.

Using the last dataframe we are going to create dictionaries to map all these values.

In [44]:
# country_code = pd.Series(countries_stock.CC.values,index=countries_stock.Ticker).to_dict()
countries = pd.Series(countries_stock.Country.values,index=countries_stock.Ticker).to_dict()
names = pd.Series(countries_stock.Name.values,index=countries_stock.Ticker).to_dict()
Sector = pd.Series(countries_stock.Sector.values,index=countries_stock.Ticker).to_dict()


stock_data["Country"] = stock_data["Ticker"].map(countries)
stock_data["Sector"] = stock_data["Ticker"].map(Sector)
stock_data["Names"] = stock_data["Ticker"].map(names)
# stock_data["CC"] = stock_data["Ticker"].map(country_code)

Staging Stock Table.

In [45]:
stock_data.head()

,Date,Ticker,Value_type,Value,Country,Sector,Names
0,2019-09-22,ABQK.QA,Adj Close,3.302143,Qatar,Financials,Ahli Bank Q.S.C.
1,2019-09-22,AHCS.QA,Adj Close,0.660800,Qatar,Industrials,Aamal Company Q.P.S.C.
2,2019-09-22,AKHI.QA,Adj Close,1.762422,Qatar,Financials,Al Khaleej Takaful Group (Q.P.S.C.)
3,2019-09-22,ALHE.TA,Adj Close,4814.037109,Israel,Real Estate,Alony Hetz
4,2019-09-22,AMOT.TA,Adj Close,2350.587646,Israel,Real Estate,Amot Investments


The final table contains more than 2.5 million rows.

In [46]:
# Number of rows
len(stock_data)

2541750

One last check to make sure there is no missing data.

In [47]:
stock_data.isnull().values.any()

False

## STAGING STRINGENCY TABLE

The file "_Covid_Stringency_Index_" collects information with the **Goverment Response Stringency Index** of each country during the pandemic.

The data is collected from the **Oxford Coronavirus Goverment Response Tracker (OxCGRT project)**. This resource is publised by researchers at the Blavatnik School of Government at the University of Oxford: Thomas Hale, Anna Petherik, Beatriz Kira, Noam Angrist, Toby Phillips and Samuel Webster.

Here you can find out more information about the data and download the complete dataset: https://ourworldindata.org/policy-responses-covid#schools-closures

_Covid_Stringency_Index_ is a composite measure based on nine response indicators including: 
- School closures
- Workplace closures
- Cancellation of public events
- Restrinctions on public gatherings
- Closures of public transport
- Stay-At-Home requirements
- Public information campaigns
- Restrictions on internal movements
- International travel controls

The index is rescaled to a value from 0 to 100. A higher score indicates a stricter government response (i.e. 100 = strictest response).
Here you can find the author's full description of how this index is calculated: https://github.com/OxCGRT/covid-policy-tracker/blob/master/documentation/index_methodology.md


In [48]:
countries_stringency = pd.read_csv("./Data/Covid_Stringency_Index.csv", delimiter=",", encoding='utf-8')

This is the structure the data has.

In [49]:
countries_stringency.head()

,Entity,Code,Date,Stringency Index (OxBSG)
0,Afghanistan,AFG,2020-01-01,0.0
1,Afghanistan,AFG,2020-01-02,0.0
2,Afghanistan,AFG,2020-01-03,0.0
3,Afghanistan,AFG,2020-01-04,0.0
4,Afghanistan,AFG,2020-01-05,0.0


The code column corresponds with the country ISO codes as described in the ISO 3166 international standard.

Because the file _Covid_Stringency_Index_ dates back to the beginning of January 2020, I'm going to expand this dataframe to match the date range of our other dataframe (since 401 days ago).

In [50]:
# Our start date in our other dataframe
start.date()

datetime.date(2019, 9, 22)

I'm going to create an auxiliar dataframe with the first date of all the countries in our _Covid_Stringency_Index_.

In [51]:
# aux_data = countries_stringency[countries_stringency['Date'] == min(countries_stringency['Date']).strftime("%Y-%m-%d")]
aux_data = countries_stringency.groupby(['Entity'], as_index = False).min()

In [52]:
aux_data.head()

,Entity,Code,Date,Stringency Index (OxBSG)
0,Afghanistan,AFG,2020-01-01,0.0
1,Albania,ALB,2020-01-01,0.0
2,Algeria,DZA,2020-01-01,0.0
3,Andorra,AND,2020-01-01,0.0
4,Angola,AGO,2020-01-01,0.0


Now I'm going to expand this dataframe back to the first date we have in our _stock_data_ dataframe, and I'm going to set their _Stringency Index_ to: "0.0" (the lowest stringency value that corresponds to a country that hasn't took any measure yet regarding covid-19).

In [53]:
prev_data = pd.concat([pd.DataFrame({'Entity' : row.Entity, 'Code': row.Code, 'Date': pd.date_range(start, aux_data['Date'].min(), freq='D').strftime("%Y-%m-%d"), 'Stringency Index (OxBSG)': 0.0}, columns=['Entity', 'Code','Date', 'Stringency Index (OxBSG)'])
          for i, row in aux_data.iterrows()], ignore_index = True)

In [54]:
prev_data

,Entity,Code,Date,Stringency Index (OxBSG)
0,Afghanistan,AFG,2019-09-22,0.0
1,Afghanistan,AFG,2019-09-23,0.0
2,Afghanistan,AFG,2019-09-24,0.0
3,Afghanistan,AFG,2019-09-25,0.0
4,Afghanistan,AFG,2019-09-26,0.0
...,...,...,...,...
18680,Zimbabwe,ZWE,2019-12-27,0.0
18681,Zimbabwe,ZWE,2019-12-28,0.0
18682,Zimbabwe,ZWE,2019-12-29,0.0
18683,Zimbabwe,ZWE,2019-12-30,0.0


All I need to do now is concatenate this dataframe with our original _Covid_Stringency_Index_ dataframe and we'll have the _STAGING STRINGENCY TABLE_ with the complete date range.

In [55]:
# Complete data
countries_stringency = pd.concat([prev_data, countries_stringency])

In [56]:
# Renaming Entity column
countries_stringency = countries_stringency.rename(columns={'Entity': 'Country', 'Stringency Index (OxBSG)': 'Stringency_Index'})

In [57]:
countries_stringency

,Country,Code,Date,Stringency_Index
0,Afghanistan,AFG,2019-09-22,0.0
1,Afghanistan,AFG,2019-09-23,0.0
2,Afghanistan,AFG,2019-09-24,0.0
3,Afghanistan,AFG,2019-09-25,0.0
4,Afghanistan,AFG,2019-09-26,0.0
...,...,...,...,...
54243,Zimbabwe,ZWE,2020-10-23,71.3
54244,Zimbabwe,ZWE,2020-10-24,71.3
54245,Zimbabwe,ZWE,2020-10-25,71.3
54246,Zimbabwe,ZWE,2020-10-26,71.3


Let's check for any missing or duplicated values.

In [58]:
countries_stringency.isnull().values.any()

False

In [59]:
countries_stringency.duplicated(subset=['Country', 'Date'], keep=False).any()

False

***

Finally, I'll upload these two staging tables to an S3 bucket to access it later on to create our database.

In [60]:
config = configparser.ConfigParser()
config.read('Cp.cfg')
KEY = config.get('AWS', 'AWS_ACCESS_KEY_ID')
SECRET =config.get('AWS','AWS_SECRET_ACCESS_KEY')
# os.environ['AWS_DEFAULT_REGION'] = 'eu-west-2'
s3 = boto3.resource('s3',
                       region_name="eu-west-3",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

In [61]:
# We can skip this step if we already have an S3 bucket created
s3.create_bucket(Bucket='capstone-project-udaz', CreateBucketConfiguration={'LocationConstraint': "eu-west-3"})

s3.Bucket(name='capstone-project-udaz')

In [62]:
# from io import StringIO 

bucket = 'capstone-project-udaz' # already created on S3
csv_buffer = StringIO()

countries_stringency.to_csv(csv_buffer, sep= ";", header = True, index = False)
s3.Object(bucket, 'Staging_stringency_table.csv').put(Body=csv_buffer.getvalue())


csv_buffer = StringIO()
stock_data.to_csv(csv_buffer, sep= ";", header = True, index = False)
s3.Object(bucket, 'Staging_stock_table.csv').put(Body=csv_buffer.getvalue())


{'ResponseMetadata': {'RequestId': 'A34C2C6516BB2301',
  'HostId': 'u9jeCNT9P2w14Xb2JGVnpdnfufydwgN5YYORNDipz8K3peeA2eDHwAsjfTPfIMklC3F9tDtK6Ro=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'u9jeCNT9P2w14Xb2JGVnpdnfufydwgN5YYORNDipz8K3peeA2eDHwAsjfTPfIMklC3F9tDtK6Ro=',
   'x-amz-request-id': 'A34C2C6516BB2301',
   'date': 'Tue, 27 Oct 2020 18:09:19 GMT',
   'etag': '"456e93465fe72fcffd35a4ec55ac9f1e"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"456e93465fe72fcffd35a4ec55ac9f1e"'}

With the data loaded in the S3 bucket we can now run the Etl.py file to create our fact and dimensional tables.

***